In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
from torch.utils import data
import torch.optim as optim

batch_size = 64
def load_data_fashion_mnist(batch_size, resize=None): 
    transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307, ), (0.3081, ))])
    mnist_train = datasets.FashionMNIST(
        root="../data", train=True, transform=transform, download=True)
    mnist_test = datasets.FashionMNIST(
        root="../data", train=False, transform=transform, download=True)
    return data.DataLoader(mnist_train, batch_size, shuffle=True),data.DataLoader(mnist_test, batch_size, shuffle=False)

train_loader,test_loader= load_data_fashion_mnist(batch_size, resize=28)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super( ResidualBlock,self).__init__()
        self.channels=channels
        self.conv1=nn.Conv2d(channels,channels,kernel_size=3,padding=1)
        self.conv2=nn.Conv2d(channels,channels,kernel_size=3,padding=1)
    def forward(self,x):
        y=F.relu(self.conv1(x))
        y=self.conv2(y)
        return F.relu(x+y)
    
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(1,16,kernel_size=3,padding=5)
        self.conv2=nn.Conv2d(16,32,kernel_size=1,padding=5)
        self.conv3=nn.Conv2d(32,1,kernel_size=1)
        self.mp=nn.MaxPool2d(2)
        
        self.rblock1=ResidualBlock(16)
        self.rblock2=ResidualBlock(32)
        
        #self.fc=nn.Linear(512,10)
        
    def forward(self,x):
        in_size=x.size(0)
        x=self.mp(F.relu(self.conv1(x)))
        x=self.rblock1(x)
        x=F.relu(self.conv2(x))
        x=self.rblock2(x)
        x=self.conv3(x)
        #x=x.view(in_size,-1)
        return x
mod=Net()

class SENet(nn.Module):
    def __init__(self):
        super(SENet,self).__init__()
        self.avepool=nn.AvgPool2d(1)
        self.conv1=nn.Conv2d(1,1,kernel_size=1)
        self.conv2=nn.Conv2d(1,1,kernel_size=1)
        self.L1=nn.Linear(784,10)
    def forward(self,x):
        in_size=x.size(0)
        x0=x
        x2=mod(x)
        x3=self.avepool(x2)
        x3=F.relu(self.conv1(x3))
        x3=self.conv2(x3)
        x2=x3*x2
        x2=x2+x0
        x2=F.relu(x2)
        x2=x2.view(in_size,-1)
        x2=self.L1(x2)
        return x2
model = SENet()
 
# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
 

def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1) # dim = 1 列是第0个维度，行是第1个维度
            total += labels.size(0)
            correct += (predicted == labels).sum().item() # 张量之间的比较运算
    print('accuracy on test set: %d %% ' % (100*correct/total))
 
 
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

D:\csrgzn\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


[1,   300] loss: 0.787
[1,   600] loss: 0.598
[1,   900] loss: 0.574
accuracy on test set: 80 % 
[2,   300] loss: 0.537
[2,   600] loss: 0.514
[2,   900] loss: 0.505
accuracy on test set: 81 % 
[3,   300] loss: 0.491
[3,   600] loss: 0.496
[3,   900] loss: 0.483
accuracy on test set: 82 % 
[4,   300] loss: 0.471
[4,   600] loss: 0.463
[4,   900] loss: 0.457
accuracy on test set: 82 % 
[5,   300] loss: 0.452
[5,   600] loss: 0.453
[5,   900] loss: 0.450
accuracy on test set: 83 % 
[6,   300] loss: 0.433
[6,   600] loss: 0.440
[6,   900] loss: 0.453
accuracy on test set: 83 % 
[7,   300] loss: 0.444
[7,   600] loss: 0.433
[7,   900] loss: 0.433
accuracy on test set: 84 % 
[8,   300] loss: 0.420
[8,   600] loss: 0.436
[8,   900] loss: 0.435
accuracy on test set: 83 % 
[9,   300] loss: 0.423
[9,   600] loss: 0.433
[9,   900] loss: 0.420
accuracy on test set: 83 % 
[10,   300] loss: 0.418
[10,   600] loss: 0.423
